# Import Package

In [1]:
#dataframe
import pandas as pd

#computation
import numpy as np

#NLP lib
import nltk

# string operations
import string 
import re

# general imports
import math


#visualization
import matplotlib.pyplot as plt

#others
import collections
from collections import Counter

#pre-build package
from utils import *
from Invertedfile_TFIDF import Invertedfile_TFIDF
from AverageWordEmbedding import AverageWordEmbedding
from WeightedAverageEmbedding import WeightedAverageEmbedding

#auto reload
%load_ext autoreload
%autoreload 2

#ignore warning
import warnings
warnings.filterwarnings('ignore')

In [16]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

# Load Data

In [2]:
df = pd.read_csv('data.tsv', sep='\t',error_bad_lines=False)
df.head()

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'


,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [3]:
df.dropna(inplace=True)
#drop rows with dupplication
df = df.drop_duplicates(subset = ['question1'], keep = 'last')
df = df.drop_duplicates(subset = ['question2'], keep = 'last')
df.nunique()

id              232203
qid1            232203
qid2            232202
question1       232203
question2       232203
is_duplicate         2
dtype: int64

### Test Set

In [4]:
df.reset_index(inplace=True)

In [5]:
#Get the first 100 "is duplicate" rows for testing
df_test = df[df['is_duplicate'] == 1].iloc[:100]
df_test.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
1,1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
6,13,11568,22332,22333.0,Which is the best book to study TENSOR for gen...,Which is the best book for tensor calculus?,1.0
11,24,33995,62359,62360.0,How does an IQ test work and what is determine...,How does IQ test works?,1.0
12,25,244506,357159,357160.0,Is it safe to use Xiaomi mobile phones?,Is it safe or unsafe to use Xiaomi Products?,1.0
14,27,375073,506056,506057.0,What are the best books on cosmology?,Which is the best book for cosmology?,1.0


# Text Preprocessing

In [6]:
df['processed_ques2'] = df.question2.apply(lambda x: text_preprocessing(x))

# TFIDF with Inverted File

In [7]:
model1 = Invertedfile_TFIDF(df[:5000])

Build vocabulary...
Build TF-IDF matrix...
Build inverted file...
Model is ready to query!


In [8]:
top2  = model1.accuracy(df_test, 2) 
top5  = model1.accuracy(df_test, 5) 

print("Accuracy with k = 2: ", top2)
print("Accuracy wit k = 5: ", top5)

Accuracy with k = 2:  0.9
Accuracy wit k = 5:  0.97


In [9]:
model1.query_engine()

Your query is: How to enjoy life
 What will you do if you fail in your life? - 2.858928385609251
-----------------------------------------------------
 What should I do now in my love life? - 2.858928385609251
-----------------------------------------------------
 Is life worth it? - 2.858928385609251
-----------------------------------------------------
 What are you searching for in your life? - 2.858928385609251
-----------------------------------------------------
 What am I missing in my life? - 2.858928385609251
-----------------------------------------------------


# Averaging Word Embedding

In [10]:
model2 = AverageWordEmbedding('glove.6B.100d.txt', 100, df[:5000])

Loading Glove Model...
Embedding All Question 2 data...
Model is ready to query


In [11]:
top2  = model2.accuracy(df_test, 2) 
top5  = model2.accuracy(df_test, 5) 

print("Accuracy with k = 2: ", top2)
print("Accuracy wit k = 5: ", top5)

Accuracy with k = 2:  0.87
Accuracy wit k = 5:  0.87


In [12]:
model2.query_engine()

Your query is: How to learn faster
 Would you like to share your learning experience and build a learning routine to help others learn fast and systematically? - -2.674782438461154
-----------------------------------------------------
 How do I learn anything? - -2.818097499894
-----------------------------------------------------
 Is there anything that moves faster than light? - -2.8681905806980845
-----------------------------------------------------
 What is the best way to learn to sew online? - -2.921265522334209
-----------------------------------------------------
 What makes dreadlocks grow faster than normal hair? - -2.925056448911585
-----------------------------------------------------


# TFIDF Weighted Average Word Embedding

In [13]:
model3 = WeightedAverageEmbedding('glove.6B.100d.txt', 100, df[:5000])

Build vocabulary...
Loading Glove Model...
Embedding All Question 2 data...
Model is ready to query


In [14]:
top2  = model3.accuracy(df_test, 2) 
top5  = model3.accuracy(df_test, 5) 

print("Accuracy with k = 2: ", top2)
print("Accuracy wit k = 5: ", top5)

Accuracy with k = 2:  0.65
Accuracy wit k = 5:  0.66


In [15]:
model3.query_engine()

Your query is: What is your motivation in life
 What's your motivation in life? - -1.3031774807489762
-----------------------------------------------------
 What is the best motivation book? - -1.8850125081504912
-----------------------------------------------------
 What are some good ways to manage my psychic or emotional sensitivity to other people? - -2.5647150550711424
-----------------------------------------------------
 What is life's most satisfying purpose? - -2.6455670742039836
-----------------------------------------------------
 What should I do to improve my explaining skills? - -2.683867634173915
-----------------------------------------------------
